Cargar el dataframe resultado del preprocesamiento

In [2]:
from dataloader import ETL
import pandas as pd

file_path = 'dataframe_merge.xlsx'
etl = ETL(file_path)
df = pd.read_excel(file_path)

In [3]:
df.head(5)

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,num_voted_users,...,last_update,inventory_id,store_id,last_update_x,rental_id,rental_date,customer_id,return_date,staff_id,last_update_y
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientis...,2006,1,6,0.99,86,20.99,76750,...,2020-01-25 14:40:46,1,1,2006-02-15 05:09:17,4863,2005-07-08 19:03:15,431,2005-07-11 21:29:15,2,2006-02-15 21:30:53
1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientis...,2006,1,6,0.99,86,20.99,76750,...,2020-01-25 14:40:46,1,1,2006-02-15 05:09:17,11433,2005-08-02 20:13:10,518,2005-08-11 21:35:10,1,2006-02-15 21:30:53
2,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientis...,2006,1,6,0.99,86,20.99,76750,...,2020-01-25 14:40:46,1,1,2006-02-15 05:09:17,14714,2005-08-21 21:27:43,279,2005-08-30 22:26:43,1,2006-02-15 21:30:53
3,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientis...,2006,1,6,0.99,86,20.99,76750,...,2020-01-25 14:40:46,2,1,2006-02-15 05:09:17,972,2005-05-30 20:21:07,411,2005-06-06 00:36:07,1,2006-02-15 21:30:53
4,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientis...,2006,1,6,0.99,86,20.99,76750,...,2020-01-25 14:40:46,2,1,2006-02-15 05:09:17,2117,2005-06-17 20:24:00,170,2005-06-23 17:45:00,2,2006-02-15 21:30:53


Tipos de variables

In [4]:
df.dtypes

film_id                int64
 title                object
 description          object
 release_year          int64
 language_id           int64
 rental_duration       int64
 rental_rate         float64
 length                int64
 replacement_cost    float64
 num_voted_users       int64
 rating               object
 special_features     object
last_update           object
inventory_id           int64
 store_id              int64
 last_update_x        object
rental_id              int64
 rental_date          object
 customer_id           int64
 return_date          object
 staff_id              int64
 last_update_y        object
dtype: object

Métricas descriptivas de las características

In [5]:
df.describe()

,film_id,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,num_voted_users,inventory_id,store_id,rental_id,customer_id,staff_id
count,16088.000000,16088.0,16088.0,16088.000000,16088.000000,16088.000000,16088.000000,16088.000000,16088.000000,16088.000000,16088.000000,16088.000000,16088.000000
mean,500.814582,2006.0,1.0,4.932185,2.951213,114.810977,20.265813,39241.229488,2290.506278,1.999254,8029.785306,297.216497,1.498633
std,288.077662,0.0,0.0,1.400854,1.650959,40.062412,6.091701,22430.929911,1320.203439,0.027302,4631.653412,172.432975,0.500014
min,1.000000,2006.0,1.0,3.000000,0.990000,46.000000,9.990000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,256.000000,2006.0,1.0,4.000000,0.990000,80.000000,14.990000,19050.000000,1161.000000,2.000000,4022.750000,147.750000,1.000000
50%,494.000000,2006.0,1.0,5.000000,2.990000,113.000000,20.990000,39250.000000,2285.000000,2.000000,8025.500000,296.000000,1.000000
75%,752.000000,2006.0,1.0,6.000000,4.990000,148.000000,25.990000,58650.000000,3430.000000,2.000000,12041.250000,446.000000,2.000000
max,1000.000000,2006.0,1.0,7.000000,4.990000,185.000000,29.990000,76900.000000,4581.000000,2.000000,16049.000000,599.000000,2.000000


# Preguntas

## 1. ¿Cuántas y cuál es el costo de reposición de las películas no devueltas?

In [6]:
columns =  [' return_date']
null_values = etl.null_count(df, columns)

for i in range(len(columns)):
    print('Valores faltantes columna %s son %i' %( columns[i], len(null_values[i])))

2024-11-15 17:43:04,455 -- dataloader -- DEBUG -- Valores NULL detectados
2024-11-15 17:43:04,456 -- dataloader -- INFO -- El método null_count tardó 0.0051 segundos en ejecutarse.


Valores faltantes columna  return_date son 183


En este caso la cantidad de valores 'NULL' de la columna ' return_date' se pueden interpretar como elementos que no fueron regresados, es decir, películas que se perdieron. Teniendo en cuenta que cada película tiene asociado un costo de reposición, entonces es posible calcular el costo total de reposición de películas perdidas.

In [7]:
costo_total = etl.sum_index(df, ' replacement_cost',null_values[0])

print('El costo de la reposición de las películas perdidas es de: ', round(costo_total, 2))

2024-11-15 17:43:04,461 -- dataloader -- DEBUG -- Cálculo realizado
2024-11-15 17:43:04,462 -- dataloader -- INFO -- El método sum_index tardó 0.0010 segundos en ejecutarse.


El costo de la reposición de las películas perdidas es de:  3552.17


#### Hay registros de 183 películas no devueltas, a las cuales se asocia un costo de reposición de 3552.17 unidades monetarias

## 2. ¿Cuál es el ranking de películas más rentadas?

In [8]:
df[' title'].value_counts().reset_index().head(10)

,title,count
0,FLAMINGOS CONNECTICUT,92
1,BUCKET BROTHERHOOD,34
2,ROCKETEER MOTHER,33
3,SCALAWAG DUCK,32
4,FORWARD TEMPLE,32
5,GRIT CLOCKWORK,32
6,JUGGLER HARDLY,32
7,RIDGEMONT SUBMARINE,32
8,ZORRO ARK,31
9,ROBBERS JOON,31


## 3. ¿Cuáles son los clientes que más películas han rentado?

In [9]:
df[' customer_id'].value_counts().reset_index().head(10)

,customer_id,count
0,148,46
1,526,45
2,144,42
3,236,42
4,75,41
5,197,40
6,469,40
7,257,40
8,137,39
9,468,39


## 4. ¿Cuál es el ranking de segmentos? 

In [10]:
df[' rating'].value_counts().reset_index()

,rating,count
0,PG-13,3629
1,NC-17,3293
2,PG,3212
3,R,3181
4,G,2773


replacement_cost / rental_rate

In [17]:
df_unique = df.drop_duplicates(subset=[' title'], keep='first')


df_unique.size


21032

In [16]:
df_nuevo = pd.DataFrame({
    'Título': df_unique[' title'],  # Opcional: Mantener otras columnas
    'Resultado': df[' replacement_cost'] / df[' rental_rate']  # División de columnas  
})

df_nuevo.head(10)

,Título,Resultado
0,ACADEMY DINOSAUR,21.20202
1,NaN,21.20202
2,NaN,21.20202
3,NaN,21.20202
4,NaN,21.20202
5,NaN,21.20202
6,NaN,21.20202
7,NaN,21.20202
8,NaN,21.20202
9,NaN,21.20202
